This notebook is useful for ceating training data:

-read content from files within a folder

-send each to ChatGPT with a prompt text added

-take the answer and insert it within specified text into a new file

-creates an output folder (if not present yet) and saves the created files there

-if filename already exists, add affix '-1/-2 etc.' at  the end -> allows to create several different versions for the same input file

-creates log file folder with log file

In [3]:
import os
#from google.colab import userdata

In [4]:
#!pip install openai

In [5]:
from openai import OpenAI

In [6]:
import openai

In [7]:
client = OpenAI(api_key="OPENAI_TOKEN")

In [8]:
#you must have saved your OpenAI key in google colab under 'Secrets' ('key' symbol on the left)
#api_key = userdata.get("OPENAI_TOKEN")
#openai.api_key = api_key
os.environ["OPENAI_API_KEY"] = ("") 
#enter name of OpenAI key -> better solution: set key in environment outside of notebook

In [9]:
#from google.colab import drive
#drive.mount('/content/drive')

In [16]:
#import os
#import openai

def send_to_chatgpt(input_text):
    response = openai.chat.completions.create(#
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return response

def process_and_save_file(input_file_path, prepend_text, append_text_pre, append_text_post, output_folder):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    text_to_send = prepend_text + file_content #text to be sent to ChatGPT
    whole_response = send_to_chatgpt(text_to_send)
    chatgpt_response = whole_response.choices[0].message.content #extract answer itself from response object

    #create trainingsfile content from text snippets and ChatGPT input/output
    new_file_content = append_text_pre + chatgpt_response + append_text_post + file_content + '"}'

    #create output folder/files (I think these 6 lines can be reduced!!)
    output_file_name = os.path.basename(input_file_path).replace(".txt", "-response.txt")
    output_file_path = os.path.join(output_folder, output_file_name)
    base_name = os.path.basename(input_file_path)
    # Remove .txt extension and add -response suffix
    base_name_without_extension = base_name.replace(".txt", "")
    #output_file_name = f"{base_name_without_extension}-response.txt"
    output_file_path = os.path.join(output_folder, output_file_name)

    # if input from the same file has been sent to ChatGPT several times in order to generate several different answers:
    #save several files with affix -1, -2, etc.
    counter = 1
    while os.path.exists(output_file_path):
        output_file_name = f"{base_name_without_extension}-response-{counter}.txt"
        output_file_path = os.path.join(output_folder, output_file_name)
        counter += 1
    

    with open(output_file_path, 'w', encoding='utf-8') as new_file:
        new_file.write(new_file_content)
        #print(f"File '{output_file_path}' has been created with the ChatGPT response.")
        #print(chatgpt_response)
        #optional, for creating a log file with input (file_content) sent to ChatGPT and given output in order to evaluate
        #Bug: last entry missing in the log file, to be checked
        log_folder = os.path.join(output_folder, 'log')
        os.makedirs(log_folder)
        log_file_path = os.path.join(log_folder, 'log.txt')  # create the log file path
        with open(log_file_path, 'a', encoding='utf-8') as log_file:
            log_file.write(chatgpt_response + '\n')
            log_file.write(file_content + '\n\n')

    '''doesn't stop
    log_folder_path = os.path.join(output_folder, 'log')  # Path to the new 'log' folder within 'output_folder'
    os.makedirs(log_folder_path, exist_ok=True)  # Create the 'log' folder if it doesn't already exist
    log_file_path = os.path.join(log_folder_path, 'log.txt')  # Path to 'log.txt' within the newly created 'log' folder
    #log_file_path = os.path.join(output_folder, 'log.txt')  # create the log file path
    with open(log_file_path, 'a', encoding='utf-8') as log_file:
      log_file.write(chatgpt_response + '\n')
      log_file.write(file_content + '\n\n')
      '''


def process_folder(input_folder, prepend_text, append_text_pre, append_text_post):
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            input_file_path = os.path.join(input_folder, file_name)
            output_folder_path = os.path.join(input_folder, 'output')#added
            os.makedirs(output_folder_path, exist_ok=True)#added

            process_and_save_file(input_file_path, prepend_text, append_text_pre, append_text_post, output_folder_path)



In [17]:
# Define your folder paths and texts here
#input_folder = '/content/drive/MyDrive/Jhana_Trainingfiles/curated_trainingsets_dialog/extract/'
input_folder = 'dialog-trainingdata/test/'
#output_folder is created above automatically within input_folder, if not existing

#instruction sent to ChatGPT:
prepend_text = "The following text is an answer given by a Jhana meditation assistant. Please give me a very short question from the user that would generate this answer."
#prepend_text = "The following text is a Jhana meditation instruction given by a meditation assistant. Please give me a short request from the user that would generate this meditation instruction for the user."
#prepend_text is added in front of the input text to ChatGPT

#insert into text snippets, e.g. alpaca format: https://huggingface.co/datasets/tatsu-lab/alpaca
append_text_pre = 'alpaca_example = {\n    "instruction": "You are a meditation assistant who guides the user through a Jhana meditation.",\n    "input": "'
append_text_post = '",\n    "response": "'

# Process all .txt files in the input folder and save the outputs in the output folder
process_folder(input_folder, prepend_text, append_text_pre, append_text_post)


FileExistsError: [Errno 17] File exists: 'dialog-trainingdata/test/output/log'